# Intro 2 Text Mining

In [1]:
# Importing the libraries
import numpy as np
import re
import pickle 
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import load_files
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/uzaycetin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Unpickling Movie Review dataset

In [2]:
# Unpickling dataset
X_in = open('uX.pickle','rb')
y_in = open('uy.pickle','rb')
X = pickle.load(X_in)
y = pickle.load(y_in)

In [3]:
type(X), len(X)

(list, 2000)

In [4]:
X[1]

b"good films are hard to find these days . \ngreat films are beyond rare . \nproof of life , russell crowe's one-two punch of a deft kidnap and rescue thriller , is one of those rare gems . \na taut drama laced with strong and subtle acting , an intelligent script , and masterful directing , together it delivers something virtually unheard of in the film industry these days , genuine motivation in a story that rings true . \nconsider the strange coincidence of russell crowe's character in proof of life making the moves on a distraught wife played by meg ryan's character in the film -- all while the real russell crowe was hitching up with married woman meg ryan in the outside world . \ni haven't seen this much chemistry between actors since mcqueen and mcgraw teamed up in peckinpah's masterpiece , the getaway . \nbut enough with the gossip , let's get to the review . \nthe film revolves around the kidnapping of peter bowman ( david morse ) , an american engineer working in south america

In [5]:
# Creating the corpus
corpus = []
for i in range(0, 2000):
    review = re.sub(r'\W', ' ', str(X[i]))
    review = review.lower()
    review = re.sub(r'^br$', ' ', review)
    review = re.sub(r'\s+br\s+',' ',review)
    review = re.sub(r'\s+[a-z]\s+', ' ',review)
    review = re.sub(r'^b\s+', '', review)
    review = re.sub(r'\s+', ' ', review)
    corpus.append(review)  

# Bag of Words 

In [6]:
# Creating the BOW model
#
# min_df = 3      a word should pass at least 3 doc
# max_df = 0.6    words that pass in more than %60 of words is also eliminated

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features = 2000, min_df = 3, max_df = 0.6, stop_words = stopwords.words('english'))
X = vectorizer.fit_transform(corpus).toarray()

In [7]:
X.shape

(2000, 2000)

In [8]:
import pandas as pd
words = vectorizer.vocabulary_
df = pd.DataFrame(data = list(words.keys()), index = list(words.values()) )
df.columns = ['ID']
df.head()

,ID
103,arnold
24,action
1602,since
946,late
632,films


# TF-IDF Model

In [9]:
# Creating the Tf-Idf model directly
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 2000, min_df = 3, max_df = 0.6, stop_words = stopwords.words('english'))
X = vectorizer.fit_transform(corpus).toarray()

In [10]:
X.shape

(2000, 2000)

# Train Test Split

In [11]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
text_train, text_test, sent_train, sent_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [12]:
text_train.shape

(1600, 2000)

# Training the Classifier

In [13]:
# Training the classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(text_train,sent_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

# Evaluation

In [14]:
# Testing model performance
sent_pred = classifier.predict(text_test)


from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(sent_test, sent_pred)

In [15]:
cm

array([[168,  40],
       [ 21, 171]])

In [16]:
accuracy_score(sent_test, sent_pred)

0.8475

# Save Classifier Model

In [17]:
# Saving our classifier
with open('myclassifier.pickle','wb') as f:
    pickle.dump(classifier,f)
    
# Saving the Tf-Idf model
with open('mytfidfmodel.pickle','wb') as f:
    pickle.dump(vectorizer,f)